For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1.  Start by creating a new Notebook for this assignment.

The notebook is created and the required dependencies are imported as follows.

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import re
from operator import itemgetter
import geocoder
import wget
import warnings
warnings.filterwarnings('ignore')

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In the following the steps taken to transform the data in the table on the Wikipedia page into a pandas dataframe are shown.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tbody>..</tbody> of HTML
tr_elements = doc.xpath('//tbody')

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements:
    i+=1
    name=t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append(name)
    if i == 181:
        break

In [5]:
table_rows = []
for i in range(len(col)):
    table_rows.append(itemgetter(*[1,3,5])(col[0::][i].split("\n")))

In [6]:
df_neighborhoods = pd.DataFrame(table_rows[1::], columns = table_rows[0] )

3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
df_neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
cl1_indx = np.where(df_neighborhoods.Borough != 'Not assigned')

In [9]:
df_ngh_clean = df_neighborhoods.iloc[cl1_indx]

In [10]:
df_ngh_clean.reset_index(inplace = True)

In [11]:
df_ngh_clean.drop(columns = 'index', inplace = True)

In [12]:
df_ngh_clean.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 2 in the table above.

- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [13]:
cl2_indx = np.where(df_ngh_clean.Neighborhood == 'Not assigned')

In [14]:
cl2_indx

(array([], dtype=int64),)

The array above is empty, so it is not the case here. The reason being the rows with not assigned Borough also have not assigned Neighborhood.

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

The notebook is cleaned and the steps taken to create the dataframe is cleary demonstrated above.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [15]:
df_ngh_clean.shape

(103, 3)

- Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Geocoder was not responsive, so the csv file was used to get the coordinates.
<!---
postal_code = df_ngh_clean['Postal Code']
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
--->

In [16]:
wget.download('https://cocl.us/Geospatial_data','geospatial_toronto.csv')

100% [................................................................................] 2891 / 2891

'geospatial_toronto.csv'

In [17]:
df_coord = pd.read_csv('geospatial_toronto.csv')

- In the following the two dataframes are merged/joined on Postal Code column as its values is unique for both of them.

In [18]:
df_combined = pd.merge(df_ngh_clean, df_coord, how='inner', left_on='Postal Code', right_on='Postal Code')

- Hence the following dataframe is created:

In [19]:
df_combined.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
